In [1]:
import sys; sys.path.append("../../automl/")

In [4]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.xgboost import XGBClassification
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

[W 2024-11-06 14:42:07,173] Trial 0 failed with parameters: {'max_depth': 6, 'grow_policy': 'depthwise', 'max_leaves': 311, 'gamma': 3.1203728088487304, 'subsample': 0.2403950683025824, 'colsample_bytree': 0.15227525095137953, 'colsample_bylevel': 0.8795585311974417, 'reg_lambda': 6.011150117432088, 'reg_alpha': 7.080725777960454, 'min_child_weight': 0, 'class_weight': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/ptrbelonovsky/python_venvs/defaualt_venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/Users/ptrbelonovsky/Desktop/UsefulMaterials/AlfaHack2/notebooks/models/../../automl/src/automl/model/xgboost/xgboost.py", line 493, in <lambda>
    lambda trial: self.objective(trial, X, y, metric),
  File "/Users/ptrbelonovsky/Desktop/UsefulMaterials/AlfaHack2/notebooks/models/../../automl/src/automl/model/xgboost/xgboost.py", line 442, in objective
 

In [5]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [8]:
RANDOM_SEED = 77
DATA_PATH = Path("../../data/")
CONFIG_PATH = Path("../../configs/config.yaml")
N_JOBS = 8

In [9]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [10]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc.parquet")
df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [11]:
df_train["target"].value_counts()

target
0    282256
1     17744
Name: count, dtype: int64

In [12]:
# undersample the 0 class
df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)

In [20]:
# add oof columns
oof_paths = ["../../data/models/lama_8029/oof.csv",
             "../../data/models/lgb_8055/oof.csv",
             "../../data/models/xgb_8052/oof.csv"]
df_oof = pd.read_csv(oof_paths[0])
for oof_file in oof_paths[1:]:
    temp = pd.read_csv(oof_file)
    df_oof = pd.concat([df_oof, temp], axis=1)

In [22]:
df_oof["target"] = df_train["target"]

In [11]:
ohe_cols = df_train.columns[df_train.columns.str.startswith("OneHotEncoder")].values.tolist()
oe_cols = df_train.columns[df_train.columns.str.startswith("OrdinalEncoder")].values.tolist()
te_cols = df_train.columns[df_train.columns.str.startswith("MeanTargetEncoder")].values.tolist()

In [12]:
# take target encoded columns
X_train, y_train = df_train[cfg["selected_features"] + te_cols + ohe_cols], df_train["target"]
X_test, y_test = df_test[cfg["selected_features"] + te_cols + ohe_cols], df_test["target"]

In [109]:
# take all columns
X_train, y_train = df_train.drop(columns=["target", "id", "smpl"]), df_train["target"]
X_test, y_test = df_test.drop(columns=["target", "id", "smpl"]), df_test["target"]

In [73]:
# take ordinal encoded columns
X_train, y_train = df_train[cfg["selected_features"] + oe_cols + ohe_cols], df_train["target"]
X_test, y_test = df_test[cfg["selected_features"] + oe_cols + ohe_cols], df_test["target"]

In [13]:
display(y_train.value_counts(normalize=True))
display(y_test.value_counts(normalize=True))

0    0.91851
1    0.08149
Name: target, dtype: float64

0    0.941322
1    0.058678
Name: target, dtype: float64

In [14]:
categorical_features = ohe_cols# + oe_cols

## Blend

In [30]:
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm

best_score = 0
best_pred = None
best_weights = (None, None, None)
metric = RocAuc()

#kf = StratifiedKFold(n_splits=5, random_state=self.random_state, shuffle=True)
for weights_1 in tqdm(np.arange(0, 1, 0.02)):
    for weights_2 in np.arange(0, 1 - weights_1, 0.02):
        weights_3 = 1 - weights_2 - weights_1

        pred = (weights_1 * df_oof.iloc[:, 0].values +
                weights_2 * df_oof.iloc[:, 1].values +
                weights_3 * df_oof.iloc[:, 2].values)

        score = metric(df_oof["target"], pred)
        
        if score > best_score:
            best_score = score
            best_pred = pred
            best_weights = (weights_1, weights_2, weights_3)

print(best_score)
print(best_weights)
print()

  0%|          | 0/50 [00:00<?, ?it/s]

0.8060247678088368
(0.12, 0.52, 0.36)



## Stack

In [32]:
X_train, y_train = df_oof.drop(columns="target"), df_oof["target"]

In [37]:
model = XGBClassification(n_jobs=N_JOBS)
model.tune(X_train, y_train, metric, timeout=60)
oof = model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

print(metric(y_train, oof))

[2024-11-06 14:42:18,449] - [   START    ] - Tuning XGBClassification
[2024-11-06 14:42:33,233] - [   OPTUNA   ] - Trial 0. New best score 0.8058377131193648 with parameters {'max_depth': 6, 'grow_policy': 'depthwise', 'max_leaves': 311, 'gamma': 3.1203728088487304, 'subsample': 0.2403950683025824, 'colsample_bytree': 0.15227525095137953, 'colsample_bylevel': 0.8795585311974417, 'reg_lambda': 6.011150117432088, 'reg_alpha': 7.080725777960454, 'min_child_weight': 0, 'class_weight': None, 'n_estimators': 473}
[2024-11-06 14:43:10,334] - [   OPTUNA   ] - Trial 3. New best score 0.805865274602378 with parameters {'max_depth': 8, 'grow_policy': 'lossguide', 'max_leaves': 27, 'gamma': 18.186408041575643, 'subsample': 0.3329019834400152, 'colsample_bytree': 0.6962700559185838, 'colsample_bylevel': 0.3805399684804699, 'reg_lambda': 5.200680211778108, 'reg_alpha': 5.4671027934327965, 'min_child_weight': 3, 'class_weight': None, 'n_estimators': 513}
[2024-11-06 14:43:19,412] - [   OPTUNA   ] - 5

## Model

In [16]:
metric = RocAuc()

In [17]:
model = XGBClassification(n_jobs=16)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=categorical_features)
oof = model.fit(X_train, y_train, categorical_features=categorical_features)
y_pred = model.predict(X_test)

print(metric(y_train, oof))
print(metric(y_test, y_pred))

[2024-11-06 09:19:23,754] - [   START    ] - Tuning XGBClassification
[2024-11-06 09:20:04,920] - [   OPTUNA   ] - Trial 0. New best score 0.8004664892928378 with parameters {'max_depth': 6, 'grow_policy': 'depthwise', 'max_leaves': 311, 'gamma': 3.1203728088487304, 'subsample': 0.2403950683025824, 'colsample_bytree': 0.15227525095137953, 'colsample_bylevel': 0.8795585311974417, 'reg_lambda': 6.011150117432088, 'reg_alpha': 7.080725777960454, 'min_child_weight': 0, 'class_weight': None, 'n_estimators': 819}
[2024-11-06 09:21:23,947] - [   OPTUNA   ] - Trial 1. New best score 0.8012945751647573 with parameters {'max_depth': 4, 'grow_policy': 'lossguide', 'max_leaves': 163, 'gamma': 10.495128632644757, 'subsample': 0.48875051677790415, 'colsample_bytree': 0.36210622617823773, 'colsample_bylevel': 0.6506676052501416, 'reg_lambda': 1.3949386065204183, 'reg_alpha': 2.9214464853521815, 'min_child_weight': 7, 'class_weight': 'balanced', 'n_estimators': 1998}
[2024-11-06 09:31:02,977] - [   OP

In [18]:
MODEL_NAME = "xgb_8052"
MODEL_DIR = Path(f"../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [19]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    print("Test:", metric(y_test, y_pred), file=f)

In [154]:
joblib.dump(model, "../../data/models/lgb_8055/lgb_8055.joblib")


['../../data/models/lgb_8055/lgb_8055.joblib']

In [158]:
with open("../../data/models/lgb_8055/params.yaml", "w") as f:
    yaml.dump(model.params, f)

In [161]:
with open("../../data/models/lgb_8055/score.txt", "w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    print("Test:", metric(y_test, y_pred), file=f)

In [145]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8047647204688908
0.7972289963856585


In [86]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8009657912533814
0.7932043263129364


In [47]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8036673066952209
0.7969396115051322


In [37]:
print(metric(y_train, oof))
print(metric(y_test, y_pred))

0.8042716798166517
0.7975841531905153


In [15]:
test = pd.read_parquet(DATA_PATH / "test_preproc.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + te_cols + ohe_cols])[:, 1]
test[['id', 'target']].to_csv('lgb_full_dataset.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method